In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Sun Jan  9 17:14:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Read the documentation of the document store and the document retriever in the Haystack framework.
# 2. Install Haystack framework. You may need to used this fork to get support for Polish QA models.

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
# !pip install git+https://github.com/deepset-ai/haystack.git
! pip install git+https://github.com/apohllo/haystack.git@b79c5b099b294ad5f6cf37a01e4c504b438b8018

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

## Document Store

Haystack finds answers to queries within the documents stored in a `DocumentStore`. The current implementations of `DocumentStore` include `ElasticsearchDocumentStore`, `FAISSDocumentStore`,  `SQLDocumentStore`, and `InMemoryDocumentStore`.

**Here:** We recommended Elasticsearch as it comes preloaded with features like [full-text queries](https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html), [BM25 retrieval](https://www.elastic.co/elasticon/conf/2016/sf/improved-text-scoring-with-bm25), and [vector storage for text embeddings](https://www.elastic.co/guide/en/elasticsearch/reference/7.6/dense-vector.html).

**Alternatives:** If you are unable to setup an Elasticsearch instance, then follow the [Tutorial 3](https://github.com/deepset-ai/haystack/blob/master/tutorials/Tutorial3_Basic_QA_Pipeline_without_Elasticsearch.ipynb) for using SQL/InMemory document stores.

**Hint**: This tutorial creates a new document store instance with Wikipedia articles on Game of Thrones. However, you can configure Haystack to work with your existing document stores.

### Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (e.g. in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [ ]:
import glob
import unicodedata
from pathlib import Path
import re
from elasticsearch import Elasticsearch
from pprint import pprint
from tqdm import tqdm
import random
import csv
import pandas as pd

In [ ]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2
# ! elasticsearch-7.9.2/bin/elasticsearch-plugin install pl.allegro.tech.elasticsearch.plugin:elasticsearch-analysis-morfologik:7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30
!curl -X GET "localhost:9200/"

{
  "name" : "c105d3704b2f",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "O90vjjk8Q4O2xtxQs8Ukfg",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


# 3. Configure one document store based on ElasticSearch and another document store based on Faiss supported by DPR:

- The ES store should properly process Polish documents.
- For DPR you should use enelpol/czywiesz-question and enelpol/czywiesz-context encoders.
- Warning: Make sure to used models uploaded past 21st of December 2021, since the first model version included a bug.


In [ ]:
# Connect to Elasticsearch
# document_store_faiss.delete_documents()

from haystack.document_stores import ElasticsearchDocumentStore
from haystack.document_stores import FAISSDocumentStore
document_store_elastic = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
document_store_elastic_dpr = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document_dpr")
document_store_faiss = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

## Preprocessing of documents

Haystack provides a customizable pipeline for:
 - converting files into texts
 - cleaning texts
 - splitting texts
 - writing them to a Document Store

In this tutorial, we download Wikipedia articles about Game of Thrones, apply a basic cleaning function, and index them in Elasticsearch.

In [ ]:
from haystack.nodes import ElasticsearchRetriever
retriever_elastic = ElasticsearchRetriever(document_store=document_store_elastic)

In [ ]:
from haystack.nodes import DensePassageRetriever
retriever_faiss = DensePassageRetriever(
    document_store=document_store_faiss,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="enelpol/czywiesz-context",
    use_gpu=True
)
retriever_elastic_dpr = DensePassageRetriever(
    document_store=document_store_elastic_dpr,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="enelpol/czywiesz-context",
    use_gpu=True
)

# 4. Pre-process all documents from the set of Polish bills (used in the previous exercises), but splitting them into individual articles:

- You can apply a simple heuristic that searches for Art. at the beginnign of the processed line, to identify the passages.
- Assing identifiers to the passages by combining the file name with the article id.
- There might be repeated identifiers, since we use a heuristic. You should ignore that problem - just make sure that you load only one passage with a specific id.

# 5. Load the passages from previous point to the document stores described in point 2.

In [ ]:
document_store_elastic.delete_documents()
document_store_faiss.delete_documents()

ustawy_path = '/content/drive/MyDrive/Studia/NLP/ustawy'

files = [f for f in glob.glob(ustawy_path + '/*.txt')]
artykuly = []
for filename in tqdm(files):
  # print(filename)
  fname = Path(filename).stem
  with open(filename, encoding='UTF-8') as f:
    tmp = ''
    lines = f.readlines()
    for line in lines:
      if line.lstrip().startswith("Art."):
        art_name = re.findall(r'Art. \d+[A-Za-z]*.', tmp)
        if len(art_name) > 0:
          # print(art_name[0][5:-1])
          art_content = ' '.join(tmp.split())
          artykuly.append({'content': art_content, 'meta': {'name': fname+'_'+art_name[0][5:-1]+'.txt'}})
        tmp = ''
      tmp += unicodedata.normalize("NFKD", line).replace("\n", " ").replace("\t", " ")
  # break

document_store_elastic.write_documents(artykuly)
document_store_elastic_dpr.write_documents(artykuly)
document_store_faiss.write_documents(artykuly)

100%|██████████| 1179/1179 [00:17<00:00, 68.45it/s] 


Writing Documents:   0%|          | 0/25042 [00:00<?, ?it/s]

In [ ]:
document_store_elastic_dpr.update_embeddings(retriever=retriever_elastic_dpr)
# document_store_elastic_dpr.save('/content/drive/MyDrive/Studia/NLP/Lab9/elastic_DPR')

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 25032 docs ...


Updating embeddings:   0%|          | 0/25032 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/5040 [00:00<?, ? Docs/s]

AttributeError: ignored

In [ ]:
document_store_faiss.update_embeddings(retriever=retriever_faiss) # to trwa za długo...
document_store_faiss.save('/content/drive/MyDrive/Studia/NLP/Lab9/faiss_DPR')

INFO - haystack.document_stores.faiss -  Updating embeddings for 25032 docs...


Updating Embedding:   0%|          | 0/25032 [00:00<?, ? docs/s]

/usr/local/lib/python3.7/dist-packages/haystack/modeling/data_handler/dataset.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  cur_tensor = torch.tensor([sample[t_name] for sample in features], dtype=torch.long)


Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/5040 [00:00<?, ? Docs/s]

In [ ]:
len(artykuly)

25042

In [ ]:
ustawy_path2 = '/content/drive/MyDrive/Studia/ustawy2'

files2 = [f for f in glob.glob(ustawy_path2 + '/*.txt')]
artykuly2 = []
for filename in tqdm(files2):
  # print(filename)
  fname = Path(filename).stem
  with open(filename, encoding='UTF-8') as f:
    tmp = ''
    lines = f.readlines()
    for line in lines:
      if line.lstrip().startswith("Art."):
        art_name = re.findall(r'Art. \d+[A-Za-z]*.', tmp)
        if len(art_name) > 0:
          # print(art_name[0][5:-1])
          art_content = ' '.join(tmp.split())
          artykuly2.append({'content': art_content, 'meta': {'name': fname+'_'+art_name[0][5:-1]+'.txt'}})
        tmp = ''
      tmp += unicodedata.normalize("NFKD", line).replace("\n", " ").replace("\t", " ")

100%|██████████| 536/536 [00:03<00:00, 134.88it/s]


In [ ]:
len(artykuly2)

19270

# 6. Randomly select 100 passages that do not describe an amendment (you can manually reject the amendments).

In [ ]:
random.seed(10)
randomlist = []
# f = open('/content/drive/MyDrive/Studia/NLP/Lab9/100passages.csv', 'w')
# writer = csv.writer(f)
row = ['passage_id', 'question', 'passage']
# writer.writerow(row)
for i in range(101):
  n = random.randint(0,len(artykuly2))
  passage_id = artykuly2[n].get('meta').get('name')
  passage = artykuly2[n].get('content')
  # row = [passage_id, '', passage]
  # randomlist.append(n)
  # writer.writerow(row)
# f.close()
# randomlist

# 7. Invent 30 factual questions based directly on 30 distinct passages that you have selected. The larger number of randomly selected passages should allow to skip those that are hard for inventing a question for them.

In [ ]:
pytania = pd.read_csv('/content/drive/MyDrive/Studia/NLP/Lab9/100passages.csv', encoding='UTF-8')
pytania = pytania[pytania['question'].notnull()]

In [ ]:
len(pytania),
pytania

,passage_id,question,passage
0,1997_553_345.txt,"Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega ka...","Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełoż..."
1,2004_177_21.txt,Z ilu osób składa się komisja przetargowa?,Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zam...
2,1996_465_111.txt,Do jakiej wysokości za zobowiązania spółki odpowiada komandytariusz?,Art. 111. Komandytariusz odpowiada za zobowiązania spółki wobec jej wierzy...
5,1994_591_35.txt,"Kiedy ustala się wartość majątku obrotowego, który stracił swoją przydatność?","Art. 35. 1. Wartość rzeczowych składników majątku obrotowego, które utr..."
10,2001_1441_74.txt,"Jakiej karze podlega armator, który wykonuje rybołówstwo morskie w polskich...","Art. 74. 1. Armator, który wykonuje rybołówstwo morskie w polskich obszara..."
18,2002_1689_31.txt,Kogo zwalnia się od akcyzy według zasady wzajemności?,"Art. 31. 1. Zwalnia się od akcyzy, jeżeli wynika to z porozumień międzyn..."
19,2001_1353_12.txt,Czy żołnierze przy wykonaniu czynności służbowej nie muszą się przedstawiać?,Art. 12. 1. Żołnierze Żandarmerii Wojskowej przed przystąpieniem do wykon...
20,1994_592_85.txt,Ile budżetów ma miasto na prawach powiatu?,Art. 85. 1. Miasto na prawach powiatu sporządza jeden budżet. 2. Uchwała b...
23,2001_1353_40.txt,Czy żołnierze Żandarmerii Wojskowej mogą uniemożliwiać ich identyfikację p...,"Art. 40. 1. Żandarmeria Wojskowa, wykonując czynności operacyjno-rozpozna..."
27,1997_735_44.txt,W jakim przypadku policja może dokonać przeszukania pomieszczeń w domu?,Art. 44. §1. W celu znalezienia i zatrzymania przedmiotów podlegających og...


# 8. Use the set of questions defined in the previous point to assess the performance of the document stores: 

In [ ]:
pytania.to_csv('/content/drive/MyDrive/Studia/NLP/Lab9/30questions.csv')

In [ ]:
answers = pd.DataFrame(columns=['passage_id', 'question', 'passage', 'finish_id', 'score', 'finish_question'])

ind = 0

for index, row in pytania.iterrows():
  passage_id = row.passage_id
  question = row.question
  passage = row.passage
  candidate_documents_elastic = retriever_elastic.retrieve(
    query=row.question,
    top_k=3
  )
  for d in candidate_documents_elastic:
    question_passage_id = d.meta.get('name')
    question_score = d.score
    question_passage = d.content
    row = [passage_id, question, passage, question_passage_id, question_score, question_passage]
    answers.loc[ind] = row
    ind += 1

answers.to_csv('/content/drive/MyDrive/Studia/NLP/Lab9/elasticsearch_answers_pandas.csv')

# candidate_documents_elastic = retriever_elastic.retrieve(
#     query=pytania.loc[27].question,
#     top_k=3,
# )
# for d in candidate_documents_elastic:
#   pprint(d.meta)
#   pprint(d.score)

In [ ]:
answers = pd.DataFrame(columns=['passage_id', 'question', 'passage', 'finish_id', 'score', 'finish_question'])

ind = 0

for index, row in pytania.iterrows():
  passage_id = row.passage_id
  question = row.question
  passage = row.passage
  # candidate_documents_faiss = retriever_faiss.retrieve(
  candidate_documents_faiss = retriever_elastic_dpr.retrieve(
    query=row.question,
    top_k=3
  )
  for d in candidate_documents_faiss:
    question_passage_id = d.meta.get('name')
    question_score = d.score
    question_passage = d.content
    row = [passage_id, question, passage, question_passage_id, question_score, question_passage]
    answers.loc[ind] = row
    ind += 1

answers.to_csv('/content/drive/MyDrive/Studia/NLP/Lab9/elasticsearch_dpr_answers_pandas.csv')

# candidate_documents_faiss = retriever_faiss.retrieve(
#     query=pytania.loc[0].question,
#     top_k=3
# )
# for d in candidate_documents_faiss:
#   pprint(d.meta)
#   pprint(d.score)
#   pprint(d.content)

In [ ]:
answers

,passage_id,question,passage,finish_id,score,finish_question
0,1997_553_345.txt,"Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega ka...","Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełoż...",1997_553_351.txt,0.909293,"Art. 351. Żołnierz, który uderza podwładnego lub w inny sposób narusza je..."
1,1997_553_345.txt,"Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega ka...","Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełoż...",1997_553_350.txt,0.909266,"Art. 350. § 1. Żołnierz, który poniża lub znieważa podwładnego, podlega ..."
2,1997_553_345.txt,"Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega ka...","Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełoż...",1997_553_246.txt,0.909011,"Art. 246. Funkcjonariusz publiczny lub ten, który działając na jego polece..."
3,2004_177_21.txt,Z ilu osób składa się komisja przetargowa?,Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zam...,2004_177_20.txt,0.929891,Art. 20. 1. Komisja przetargowa jest zespołem pomocniczym kierownika zamawia...
4,2004_177_21.txt,Z ilu osób składa się komisja przetargowa?,Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zam...,2004_177_21.txt,0.929721,Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zam...
...,...,...,...,...,...,...
85,1997_153_112.txt,Kto wybiera przewodniczącego Rady Krajowego Związku Kas?,Art. 112. Rada Krajowego Związku Kas wybiera spośród swoich członków prz...,1997_153_107.txt,0.903427,Art. 107. 1. Krajowy Związek Kas zrzesza regionalne i branżowe kasy. 2. Si...
86,1997_153_112.txt,Kto wybiera przewodniczącego Rady Krajowego Związku Kas?,Art. 112. Rada Krajowego Związku Kas wybiera spośród swoich członków prz...,1997_153_118.txt,0.903024,"Art. 118. 1. Zarząd kieruje działalnością Krajowego Związku Kas, z zastr..."
87,1996_465_302.txt,Na jakiej wartości akcje dzieli się kapitał spółki akcyjnej?,Art. 302. Kapitał zakładowy spółki akcyjnej dzieli się na akcje o równej ...,1996_465_437.txt,0.931051,Art. 437. §1. Zapis na akcje sporządza się w formie pisemnej na formularzu...
88,1996_465_302.txt,Na jakiej wartości akcje dzieli się kapitał spółki akcyjnej?,Art. 302. Kapitał zakładowy spółki akcyjnej dzieli się na akcje o równej ...,1996_465_329.txt,0.929921,Art. 329. §1. Akcjonariusz obowiązany jest do wniesienia pełnego wkładu na ...


## Initalize Retriever, Reader,  & Pipeline

### Retriever

Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question could be answered.
They use some simple but fast algorithm.

**Here:** We use Elasticsearch's default BM25 algorithm

**Alternatives:**

- Customize the `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `DensePassageRetriever` to use different embedding models for passage and query (see Tutorial 6)

In [ ]:
from haystack.nodes import ElasticsearchRetriever
retriever_elastic = ElasticsearchRetriever(document_store=document_store_elastic)

In [ ]:
from haystack.nodes import DensePassageRetriever
retriever_faiss = DensePassageRetriever(
    document_store=document_store_faiss,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

/content/drive/MyDrive/Studia/NLP/ustawy/2004_1291.txt


In [ ]:
# artykuly

In [ ]:
# Alternative: An in-memory TfidfRetriever based on Pandas dataframes for building quick-prototypes with SQLite document store.

# from haystack.nodes import TfidfRetriever
# retriever = TfidfRetriever(document_store=document_store)

# 11. Compare the performance of the data stores using the following metrics: Pr@1, Rc@1, Pr@3, Rc@3.

In [ ]:
# wszystko jest w excelach w razie czego


'''
2 - to dokładnie nasz dokument
1 - to inny poprawny dokument (zatwierdzony przeze mnie)
0 - błędny dokument

True Positite to trafiona 2 (na 1 miejscu dla @1 albo gdziekolwiek dla @3)
False Negative to 0 zamiast 2 (na 1 miejscu dla @1 albo gdziekolwiek dla @3)
False Positive to 1 zamiast 2 (na 1 miejscu dla @1 albo gdziekolwiek dla @3)

FAISS - wyniki
| Pr@1 | Rc@1 | Pr@3 | Rc@3 |
| 0.8  |0.1333|0.5714|0.3333|

Elastic - wyniki
| Pr@1 | Rc@1 | Pr@3 | Rc@3 |
|0.7619|0.5333| 0.92 |0.8214|

W obu przypadkach dla @1 jak coś było przewidywane jako pozytywna klasa to raczej poprawnie przewidywało poprawny dokument (precision)
FAISS często nie trafiał dla @1 w poprawną odpowiedź (recall) a Elastic generalnie sobie radził, bo około połowę dobrze robił.
Dla @3 FAISS obniżył precyzję a zwiększył recall, czyli można powiedzieć, że zaczął trafiać w @3 poprawne dokumenty, ale i tak generalnie mało trafiał. 
Dla @3 Elastic zwiększył oba parametry, co można uznać za sukces.
'''

# 12. Make a pull request to the repository: https://github.com/apohllo/simple-legal-questions-pl containing the file with the questions.

# 13. Answer the following questions:

- Which of the document stores performs better? Take into account the different metrics enumerated in the previous point. \
Biorąc pod uwage metryki to  generalnie Elasticsearch poradził sobie lepiej. Ale wydaje mi się że to dlatego, że układałem na tyle specyficzne pytania, że mogły one zaweirać słowa po których elastic łatwo szukał odpowiedzi. Sporo tekstów i pytań było dość dziwacznych. Generalnie DR chyba powinien dac lepsze wyniki, więc możliwe, że za małym korpusem go nakarmiliśmy.
- Which of the document stores is faster?\
ElasticSearch był zdecydowanie szybszy, przynajmniej jeżeli chodzi o przygotowanie dokumentów. Przy 30 pytaniach i odpytywaniu queryiesów raczej nie było różnicy.
- Try to determine the other pros and cons of using sparse and dense document retrieval models.\
Poza wspomnianą różnicą w szybkości działaia... 
Dense Retrieval (DR) wolno się uczą i dają sporo negatywnych wyników batchach. DR ma za to tą swobodę, że może uzywać różnych modeli do wyciągania informacji, więc jak nauczymy lepszy model to będzie lepsze wyniki dawał, ale to jest też minus bo SR można używać od razu. 
Sparse Retreival (SR) jeżeli dobrze rozumiem ideę nie możemy uczyć, a DR można do konkretnego celu wytrenować. 
SR nie potrafi odróżnić słów semantycznie podobnych jak "cześć", "hej", "witam", a DR już się tego powinien móc nauczyć.

